
### Non-negative Matrix Factorization (NMF) using Alternating Least Squares (ALS)

The goal of NMF is to approximate a given matrix \( A \) as the product of two lower-dimensional matrices \( W \) and \( H \), such that all these matrices have non-negative elements. Mathematically, this can be represented as \( A \approx W \times H \).

### Algorithm Steps

1. **Initialization**: Start with random non-negative matrices \( W \) and \( H \) as initial approximations.

2. **Iterative Optimization**: The algorithm then enters a loop, where it alternates between two steps until a stopping criterion is met (e.g., maximum number of iterations, or the approximation error falls below a certain threshold).

    - **Step 1 - Update \( H \) given \( W \)**: While keeping \( W \) fixed, update \( H \) to better approximate \( A \). This is done by solving a least squares problem to minimize the difference \( A - W \times H \).
    
    - **Step 2 - Update \( W \) given \( H \)**: Similarly, while keeping \( H \) fixed, update \( W \). Again, this is done by solving a least squares problem.

3. **Check for Convergence**: After each pair of updates, check how well \( W \times H \) approximates \( A \). If the approximation is good enough, stop the algorithm.

4. **Result**: The final \( W \) and \( H \) matrices are used as the approximation of \( A \), i.e., \( A \approx W \times H \).


In [2]:
import numpy as np

np.random.seed(1)
m , n = 9 , 7
A = np.random.rand(m,n)
A


array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
        1.46755891e-01, 9.23385948e-02, 1.86260211e-01],
       [3.45560727e-01, 3.96767474e-01, 5.38816734e-01, 4.19194514e-01,
        6.85219500e-01, 2.04452250e-01, 8.78117436e-01],
       [2.73875932e-02, 6.70467510e-01, 4.17304802e-01, 5.58689828e-01,
        1.40386939e-01, 1.98101489e-01, 8.00744569e-01],
       [9.68261576e-01, 3.13424178e-01, 6.92322616e-01, 8.76389152e-01,
        8.94606664e-01, 8.50442114e-02, 3.90547832e-02],
       [1.69830420e-01, 8.78142503e-01, 9.83468338e-02, 4.21107625e-01,
        9.57889530e-01, 5.33165285e-01, 6.91877114e-01],
       [3.15515631e-01, 6.86500928e-01, 8.34625672e-01, 1.82882773e-02,
        7.50144315e-01, 9.88861089e-01, 7.48165654e-01],
       [2.80443992e-01, 7.89279328e-01, 1.03226007e-01, 4.47893526e-01,
        9.08595503e-01, 2.93614148e-01, 2.87775339e-01],
       [1.30028572e-01, 1.93669579e-02, 6.78835533e-01, 2.11628116e-01,
        2.65546659e-01, 4


Doğrusal en küçük kareler problemi çözmek için \( x \) vektörünü şu formül ile hesaplarız:

\[
x = (A^T A)^{-1} A^T b
\] === np.lingalg.lstsq

Bu formül, \( Ax = b \) denklem sisteminin "en iyi" çözümünü verir, yani \( b - Ax \) fark vektörünün 2-normunu minimize eder:

\[
\| b - Ax \|_2 = \min
\]


In [3]:
from numpy.linalg import norm

def nmf_als(A, k, max_iter=100, tol=1e-4):
    m, n = A.shape
    
    np.random.seed(42)
    W = np.random.rand(m, k)
    H = np.random.rand(k, n)
    
    for i in range(max_iter):
        H = np.linalg.lstsq(W.T @ W, W.T @ A, rcond=None)[0]
        H[H < 0] = 0  # Ensure non-negativity
        W = np.linalg.lstsq(H @ H.T, H @ A.T, rcond=None)[0].T
        W[W < 0] = 0  
        
        A_approx = W @ H
        error = norm(A - A_approx, 'fro')
        
        if error < tol:
            break
    
    return W, H

# Apply NMF using ALS
k = 3
W, H = nmf_als(A, k)

W, H


(array([[0.77613009, 0.        , 0.27925326],
        [0.86458446, 0.1286295 , 0.4922414 ],
        [0.85642126, 0.12561144, 0.02846901],
        [0.42126053, 0.        , 2.05769762],
        [1.69352101, 0.06600393, 0.        ],
        [1.13279505, 0.24337856, 0.        ],
        [1.37783299, 0.        , 0.36453854],
        [0.        , 0.1270306 , 0.5124278 ],
        [0.14989713, 0.15820555, 0.94693284]]),
 array([[0.12126163, 0.58946884, 0.        , 0.21758702, 0.52802461,
         0.15867871, 0.29047846],
        [0.28616528, 0.05607219, 3.17802122, 0.2706672 , 0.        ,
         2.56046692, 2.71149693],
        [0.45148645, 0.03553885, 0.30679405, 0.42568156, 0.28380505,
         0.        , 0.        ]]))